In [52]:
import requests
import json
import csv

In [42]:
def get_url(layerDefs, returnIdsOnly = False):
    '''
        Function to get query url for rest api
    '''
    return f'https://arcgis.metc.state.mn.us/server/rest/services/EnterpriseLibrary/TransitStops/FeatureServer/query?layerDefs={layerDefs}&returnIdsOnly={str(returnIdsOnly).lower()}&f=json'

In [66]:
# Get all object ids at this endpoint
layerDefs = '{"0":"OBJECTID>0"}'
response = json.loads(requests.post(get_url(layerDefs, returnIdsOnly=True)).text)
objectIds = response['layers'][0]['objectIds']
len(objectIds)

20874

In [81]:
# Iterate over 1500 objects at a time (max limit: 2000)
stations = []
for i in range(int(len(objectIds) / 1500 + 1)):
    min = i * 1500
    max = (i + 1) * 1500
    layerDefs = '{"0":"OBJECTID>' + str(min) + ' AND OBJECTID<' + str(max) + '"}'
    object_dict = json.loads(requests.post(get_url(layerDefs, returnIdsOnly=False)).text)
    stations.extend([s['attributes'] for s in object_dict['layers'][0]['features']])

In [82]:
# Get columns
columns = {k for s in stations for k, v in s.items()}

In [83]:
# Save as CSV
with open('MN_station_metadata.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(columns)
    for station in stations:
        csvwriter.writerow([station[c] for c in columns])